In [ ]:

for file_name in os.listdir(dataset_path):
    img_path = os.path.join(dataset_path, file_name)
    image = face_recognition.load_image_file(img_path)
    encodings = face_recognition.face_encodings(image)
    if encodings:
        known_face_encodings.append(encodings[0])
        known_face_names.append(os.path.splitext(file_name)[0])

print(f"✅ Loaded {len(known_face_encodings)} known faces.")

# Load email CSV
email_dict = {}
with open(email_data_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        email_dict[row[0]] = row[1]

# Send Email
def send_email_alert(name, email):
    subject = "Alcohol Consumption Alert!"
    body = f"Hello sir/mam,\n\n{name} has consumed alcohol."

    msg = MIMEMultipart()
    msg["From"] = EMAIL_ADDRESS
    msg["To"] = email
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))

    try:
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        server.sendmail(EMAIL_ADDRESS, email, msg.as_string())
        server.quit()
        print(f"📧 Email sent to {name} ({email}) successfully!")
    except Exception as e:
        print(f"❌ Error sending email to {name}: {e}")

# Connect to Arduino
try:
    arduino = serial.Serial(port='COM5', baudrate=9600, timeout=1)
    time.sleep(2)
    print("🔌 Connected to Arduino on COM5.\n")
except Exception as e:
    print(f"❌ Could not connect to Arduino: {e}")
    exit()

# Video capture
video_capture = cv2.VideoCapture(0)
start_time = time.time()
recognized_faces = set()

while True:
    ret, frame = video_capture.read()
    if not ret:
        print("❌ Failed to capture frame. Exiting.")
        break

    # Read from Arduino
    sensor_value = None
    if arduino.in_waiting > 0:
        try:
            data = arduino.readline().decode('utf-8').strip()
            if data.isdigit():
                sensor_value = int(data)
                print(f"📟 Sensor Value: {sensor_value}")
        except Exception as e:
            print(f"⚠️ Error reading sensor data: {e}")

    # Resize frame
    small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    rgb_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    # Detect faces
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)

        recognized_name = "Unknown"
        if face_distances[best_match_index] < 0.45:
            recognized_name = known_face_names[best_match_index]

        top, right, bottom, left = top * 2, right * 2, bottom * 2, left * 2
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, recognized_name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

        # Email logic
        if recognized_name in email_dict and recognized_name not in recognized_faces:
            if sensor_value is not None and sensor_value > 400:
                send_email_alert(recognized_name, email_dict[recognized_name])
                recognized_faces.add(recognized_name)
                time.sleep(3)
            else:
                print(f"⚠️ {recognized_name} detected but sensor value too low or not read.")

    cv2.imshow('Face Recognition', frame)

    if time.time() - start_time > 10:
        print("✅ Face recognition completed.")
        break

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("❌ User terminated program. Exiting...")
        break

# Cleanup
video_capture.release()
arduino.close()
cv2.destroyAllWindows()
print("🔒 Resources released.");
